#### Import Julia packages

In [ ]:
# import Pkg
# Pkg.add("JSON3")
# Pkg.add("Interpolations")

In [ ]:
using Gridap
using GridapMakie, CairoMakie, FileIO
using Gridap.FESpaces
using Gridap.ReferenceFEs
using Gridap.Arrays
using Gridap.Algebra
using Gridap.Geometry
using Gridap.Fields
using Gridap.CellData
using FillArrays
using Test
using InteractiveUtils
using PyPlot
using JSON3
using Dates
using Interpolations

#### Import Custom functions

In [ ]:
include("Julia_functions/indicator_chi.jl")
include("Julia_functions/cost_fun.jl")
include("Julia_functions/gradient_descent.jl")
include("Julia_functions/solvers.jl")
include("Julia_functions/find.jl")
include("Julia_functions/res_plot.jl")
include("Julia_functions/get_domain.jl")
include("Julia_functions/get_price_temp.jl")
include("Julia_functions/post_processing.jl")

#### Read JSON File

In [ ]:
width, height, windows, doors, heating_elem = get_room("Boundary.json")

In [ ]:
price_EUR, price_NOK, T_out, hour_lst = get_price_temp("WeatherandEnergy.json")

#### Boundary conditions

In [ ]:
f_Γ_w = generate_f_Γ(windows, width, height)
f_Γ_d = generate_f_Γ(doors, width, height)
f_Γ_wall(x) = !(f_Γ_w(x) || f_Γ_d(x))

#### Domain & Discretization

In [ ]:
domain = (0, width , 0, height)
partition = (25,25)
model = CartesianDiscreteModel(domain,partition) 
update_labels!(1, model, f_Γ_w, "window")
update_labels!(2, model, f_Γ_d, "door")
update_labels!(3, model, f_Γ_wall, "wall")

order = 1
degree = 2*order

Ω = Triangulation(model)
dΩ = Measure(Ω, degree)

Γ_w = BoundaryTriangulation(model, tags="window")
dΓ_w = Measure(Γ_w, degree)

Γ_d = BoundaryTriangulation(model, tags="door")
dΓ_d = Measure(Γ_d, degree)

Γ_wall = BoundaryTriangulation(model, tags="wall")
dΓ_wall = Measure(Γ_wall, degree)

reffe = ReferenceFE(lagrangian,Float64,order)
Testspace = TestFESpace(model,reffe,conformity=:H1)                         
Trialspace = TransientTrialFESpace(Testspace)                                
Uspace = FESpace(model, reffe, conformity=:H1)

# fig = CairoMakie.plot(Ω)
# scatter!(Ω, marker=:circle, markersize=20, color=:blue) 
# display(fig)

#### Time parameters

In [ ]:
t0 = 0.0    # Start time
tF = 28000/2   # End time
Δt = 10.0   # Timestep

#### Room parameters

In [ ]:
timescale = 2
h_wall(x) = 0.22*timescale*0.05            # [W/m^2K]
h_window(x) = 1.2*timescale*0.05
h_door(x) = 1.0*timescale*0.05

ρ(x)=1.225
c(x)=1020.0
k(x)=15*timescale

h = (h_wall, h_window, h_door) 
constants = (c, ρ, k, h) 

#Toutdoor(x,t)=273.0
#Tout(t)=x->Toutdoor(x,t)

Tout = get_temp_function(T_out, tF)

#price = get_price_function(price_NOK, tF, 1/timescale*0.1)
price = t->1-χ(t,2,8)

Tini(x)=297.0

TIni=interpolate_everywhere(Tini, Uspace(t0))
Tfin=interpolate_everywhere(297.0, Uspace(tF))

#### Controll parameter

In [ ]:
#q_pos(x) = χ(x[1], 0.21, 1.73) * χ(x[2], 0.13, 0.16)
q_pos = heating_elements(heating_elem)
Q(x,t)=χ(t,0.0,100.0)*3000.0*q_pos(x)*0.0
Qt(t)=x->Q(x,t)

#### Solver parameters

In [ ]:
L2norm(u)=√((tF-t0)*∑(Δt*∑(∫(uu⋅uu)*dΩ) for (t,uu) in u))
L2skp(u)=(tF-t0)*∑(Δt*∑(∫(uu⋅uu)*dΩ) for (t,uu) in u)

γ = 5000.0

Proj(a,b,z) = min(max(a,z),b)
a=0.0
b=1000.0*timescale
Proj(z) = [(t,FEFunction(Uspace,map(x->Proj(a,b,x), get_free_dof_values(zz)))) for (t,zz) in z]

#### Test Gradient DEs

In [ ]:
(Ts,qs,Ws,costs) = GradientDescent(;solveSE=SEsolver, 
solveAE=AEsolver, 
spaces=(Trialspace, Testspace, Uspace), 
dΩ=dΩ, 
dΓ=(dΓ_w,dΓ_d,dΓ_wall), 
Q=Qt, 
J=E, 
∇f=∇e, 
P=Proj, 
s_min=s_min,
sminargs=nothing, 
saveall=false, 
tol=1e-5, 
iter_max=3,
armijoparas=(ρ=0.1, α_0=1, α_min=1e-5, σ=0.0), 
Δt=Δt, 
t0=t0,
tF=tF,
Tout=Tout,
constants=constants,
Tfin=Tfin,
q_pos=q_pos)

In [ ]:
# Opprett en mappe for midlertidige filer hvis den ikke eksisterer
if !isdir("tmp")
    mkdir("tmp")
end

# Initialtilstanden og resten av tidsløsningene
# uh0 = solution[1][2]  # Første element er (t0, T0), vi henter T0
# uh  = solution[2:end]  # De resterende løsningene


uh = Ts
# Lagre resultater i Paraview-format (VTK)
createpvd("results") do pvd
    # Første løsning (t=0)
    # pvd[0] = createvtk(Ω, "tmp/results_0.vtu", cellfields=["u" => uh0])

    # Lagrer løsninger for hver tidssteg
    for (tn, uhn) in uh
        pvd[tn] = createvtk(Ω, "tmp/results_$tn.vtu", cellfields=["u" => uhn])
    end
end


In [ ]:
T = get_temperature_int(Ts, width, height)
q = get_control_int(qs)
result_plot()